In [1]:
from src.data.generate_dataset import generate_balanced_dataset
from src.data.generate_noise import generate_noise_dataset, generate_noise_image
from src.data.generate_image import generate_image

# Generate balanced dataset #

In [2]:
DATASET_PATH = "../data/interim/generated/frames1/"
NOISE_PATH = "../data/interim/generated/noise1/"

RAW_FRAMES_PATH = "../data/raw/frames"

In [8]:
generate_balanced_dataset(path=DATASET_PATH, n_copies=3,epsilon_range=(0.0, 0.1), noise_path=NOISE_PATH, seed=12)

 52%|█████▏    | 52/100 [00:10<00:09,  5.20it/s]


KeyboardInterrupt: 

In [5]:
generate_balanced_dataset(path=DATASET_PATH, n_copies=1, epsilon_range=(0.0, 0.1), seed=12)

100%|██████████| 100/100 [00:09<00:00, 10.41it/s]


In [24]:
import numpy as np

In [28]:
len(np.arange(0.0, 0.5, 0.002))

250

In [4]:
generate_balanced_dataset(path = DATASET_PATH, n_copies=25, noise_path=NOISE_PATH, zipfile=True, filename="data25copies.zip")

 20%|█▉        | 196/1000 [09:51<40:26,  3.02s/it]  


KeyboardInterrupt: 

In [20]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import numpy as np
from PIL import Image

In [3]:
import pandas as pd

parameters = pd.read_csv(DATASET_PATH+"parameters.csv", index_col=False)

In [9]:
parameters.head()

,width,height,epsilon,ring_center_width,ring_center_height,min_brightness,max_brightness,filename
0,640,480,0.000,323,240,80,210,00000.png
1,640,480,0.001,324,243,80,210,00001.png
2,640,480,0.002,324,237,80,210,00002.png
3,640,480,0.003,317,244,80,210,00003.png
4,640,480,0.004,326,239,80,210,00004.png


In [71]:
class ImageDataset(Dataset):
    def __init__(self, path: str, parameters: pd.DataFrame, transform=None) -> None:
        self.image_paths = np.array([path + filename for filename in parameters['filename'].to_numpy()])
        self.labels = parameters['epsilon'].to_numpy()
        self.transform = transform
    
    def __getitem__(self, inx):
        image_path = self.image_paths[inx]
        label = self.labels[inx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image, label
        
    def __len__(self):
        return len(self.image_paths)

In [72]:
transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [65]:
image_dataset = ImageDataset(path=DATASET_PATH, parameters=parameters, transform=transform)

In [89]:
import math
from typing import Tuple
from numpy.linalg import matrix_power

In [87]:
y, x = np.indices([480, 640])

In [108]:
new = generate_pure_image(size=(640,480), epsilon=0.5, ring_center=(320, 240), brightness=(80,210))

In [115]:
old = generate_pure_image(size=(640,480), epsilon=0.5, ring_center=(320, 240), brightness=(80,210))

In [116]:
np.array_equal(old, new)

True